# Capstone Project Notebook

This notebook contains the capstone project of the IBM Data Science Professional Certificate.

## Segmenting and Clustering Neighborhoods in Toronto

Clustering neighborhoods of Toronto city.

In [1]:
#libraries needed 

import pandas as pd
import numpy as np
import requests
import json 
import geopy
import folium 
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

### 1. Using web scraping to extract data about the postal code and neighborhoods of Toronto from Wikipedia.

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url).text

In [3]:
#parsing
soup = BeautifulSoup(html_data, 'html.parser')

In [4]:
#creating df from the web scraping 
toronto_postal = pd.DataFrame(columns=["Postal code", "Borough", "Neighborhood"])

for row in soup.find("tbody").find_all("tr")[1:]:
    col = row.find_all("td")
    code = col[0].text.replace('\n', '')
    bor = col[1].text.replace('\n', '')
    neig = col[2].text.replace('\n', '')
    
    toronto_postal = toronto_postal.append({"Postal code":code, "Borough":bor, "Neighborhood" : neig}, ignore_index=True)

In [5]:
toronto_postal.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
# Ignore cells with a borough that is Not assigned
toronto_postal["Borough"].replace("Not assigned", np.nan, inplace=True)
toronto_postal.dropna(subset = ["Borough"] , inplace = True)

In [7]:
# If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough
a = toronto_postal["Neighborhood"] == "Not assigned"
a.unique()

array([False])

In [8]:
toronto_postal.shape

(103, 3)

### 2. Appending Latitude and Longitude to the dataframe

I´m going to use the csv file with the latitude and longitude values due to it was not possible to retrieve this information through the Geocoder package.

In [9]:
#loading latlog file
from google.colab import files
import io
uploaded = files.upload()

Saving Geospatial_Coordinates.csv to Geospatial_Coordinates (2).csv


In [10]:
latlog = pd.read_csv(io.BytesIO(uploaded['Geospatial_Coordinates.csv']))

In [11]:
latlog.rename(columns={"Postal Code":"Postal code"}, inplace = True)
df = pd.merge(left=toronto_postal, right=latlog, on ="Postal code")
df


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


### 3. Explore and cluster the neighborhoods in Toronto. 

I´m going to work with only one borough, namenly: downtown toronto; and then replicate the same analysis we did to the New York City data.

In [12]:
downTor = df[df["Borough"] == 'Downtown Toronto']
downTor.reset_index(inplace = True)
downTor.drop(["index"], axis = 1)


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


Retrieving geographical coordinates of Downtown Toronto

In [13]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="dt_explorer")
location = geolocator.geocode(address)
latitudeDT = location.latitude
longitudeDT = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitudeDT, longitudeDT))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


Map of Toronto with downtown neighborhoods superimposed on top. As we have been working with the postal codes, these will be the points plotted on the map.

In [14]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitudeDT, longitudeDT], zoom_start=12)

# add markers to map
for lat, lng, label in zip(downTor['Latitude'], downTor['Longitude'], downTor['Postal code']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto 

Defining Foursquare Credentials and Version

In [15]:
CLIENT_ID = 'PZNM1XLH0YX4P4I3' # your Foursquare ID
CLIENT_SECRET = 'UHWBV3KVSBAS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Let´s explore the first postal code and its neighborhoods

In [16]:
downTor.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

Let's get the top 100 venues that are in Regent Park and Harbourfront within a radius of 500 meters.

In [17]:
# Creating the url for the get request 
lat1 = downTor.loc[0, "Latitude"]
log1 = downTor.loc[0, "Longitude"]

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat1, 
    log1, 
    radius, 
    LIMIT)

In [18]:
#Get request
results = requests.get(url).json()

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
#Cleaning the json and structure it into a _pandas_ dataframe.

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [21]:
# How many venues were returned by Foursquare?
nearby_venues.shape[0]

44

#### 3.1. Explore all Postal codes and Neighborhoods in Downtown Toronto

Let's create a function to repeat the same process to all the neighborhoods in the Downtown.

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal code', 
                  'Postal code Latitude', 
                  'Postal code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
#Using the above function to retrieve the venues for all the postal codes
downtownToronto_venues = getNearbyVenues(names = downTor['Postal code'],
                                         latitudes = downTor['Latitude'],
                                         longitudes = downTor['Longitude']
                                         )

M5A
M7A
M5B
M5C
M5E
M5G
M6G
M5H
M5J
M5K
M5L
M5S
M5T
M5V
M4W
M5W
M4X
M5X
M4Y


In [25]:
#checking the output
print(downtownToronto_venues.shape)
downtownToronto_venues.head()

(1227, 7)


,Postal code,Postal code Latitude,Postal code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [26]:
#checking how many venues were returned for each p.code
downtownToronto_venues.groupby('Postal code').count().iloc[:,1]

Postal code
M4W      4
M4X     44
M4Y     80
M5A     44
M5B    100
M5C     80
M5E     58
M5G     61
M5H     93
M5J    100
M5K    100
M5L    100
M5S     32
M5T     68
M5V     16
M5W     99
M5X    100
M6G     16
M7A     32
Name: Postal code Longitude, dtype: int64

In [27]:
#Cheking how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(downtownToronto_venues['Venue Category'].unique())))

There are 210 uniques categories.


Analyzing each postal code

In [28]:
# one hot encoding
Toronto_onehot = pd.get_dummies(downtownToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal code column back to dataframe
Toronto_onehot['Postal code'] = downtownToronto_venues['Postal code'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Postal code,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,...,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
#new dataframe size
Toronto_onehot.shape

(1227, 211)

Grouping rows by postal code and by taking the mean of the frequency of occurrence of each category

In [30]:
toronto_grouped = Toronto_onehot.groupby('Postal code').mean().reset_index()
toronto_grouped

,Postal code,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,...,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4W,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.25,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
1,M4X,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.022727,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000000,0.045455,0.022727,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.068182,0.00,0.0000,0.000000,0.000000,0.022727,0.00,0.0000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.00,0.000000,0.00,0.000000,0.022727,0.00,0.000000,0.022727,0.000000,0.0000,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.022727
2,M4Y,0.0125,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012500,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.012500,0.000000,0.000000,0.000000,0.000000,0.0000,0.012500,0.0000,0.000000,0.012500,0.000000,0.012500,0.000000,0.025000,...,0.037500,0.00,0.0125,0.000000,0.012500,0.000000,0.00,0.0125,0.000000,0.000000,0.00,0.00,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.012500,0.0125,0.00,0.062500,0.00,0.000000,0.000000,0.00,0.000000,0.012500,0.012500,0.0125,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.025000
3,M5A,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.022727,0.00,0.022727,0.000000,0.000000,0.022727,0.0000,0.0000,0.0000,0.000000,0.068182,0.022727,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.045455,0.022727,0.000000,0.000000,0.000000,...,0.045455,0.00,0.0000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.022727,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.00,0.000000,0.0000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.045455,0.0000,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.022727
4,M5B,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.020000,0.0000,0.000000,0.000000,0.000000,0.030000,0.000000,0.010000,...,0.010000,0.00,0.0000,0.000000,0.000000,0.010000,0.00

Printing each postal code along with the top 5 most common venues

In [31]:
num_top_venues = 5

for pcode in toronto_grouped['Postal code']:
    print("----"+pcode+"----")
    temp = toronto_grouped[toronto_grouped['Postal code'] == pcode].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4W----
                 venue  freq
0                 Park  0.50
1           Playground  0.25
2                Trail  0.25
3  Moroccan Restaurant  0.00
4  Martial Arts School  0.00


----M4X----
         venue  freq
0         Café  0.07
1   Restaurant  0.07
2  Coffee Shop  0.07
3       Bakery  0.05
4          Pub  0.05


----M4Y----
                 venue  freq
0          Coffee Shop  0.09
1     Sushi Restaurant  0.06
2  Japanese Restaurant  0.06
3              Gay Bar  0.04
4           Restaurant  0.04


----M5A----
            venue  freq
0     Coffee Shop  0.16
1            Park  0.07
2          Bakery  0.07
3  Breakfast Spot  0.05
4         Theater  0.05


----M5B----
             venue  freq
0   Clothing Store  0.09
1      Coffee Shop  0.08
2  Bubble Tea Shop  0.03
3             Café  0.03
4   Cosmetics Shop  0.03


----M5C----
                 venue  freq
0                 Café  0.06
1          Coffee Shop  0.06
2         Cocktail Bar  0.04
3  American Restaurant  0.04
4    

Putting the above tables in a single data frame

In [32]:
#function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
# new dataframe and display the top 10 venues for each postal code
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postalcode_venues_sorted = pd.DataFrame(columns=columns)
postalcode_venues_sorted['Postal code'] = toronto_grouped['Postal code']

for ind in np.arange(toronto_grouped.shape[0]):
  postalcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postalcode_venues_sorted.head()

,Postal code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
1,M4X,Coffee Shop,Restaurant,Café,Pub,Italian Restaurant,Bakery,Pizza Place,Market,Caribbean Restaurant,Butcher
2,M4Y,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Pizza Place,Men's Store,Mediterranean Restaurant,Hotel
3,M5A,Coffee Shop,Bakery,Park,Café,Theater,Pub,Breakfast Spot,Restaurant,Electronics Store,Event Space
4,M5B,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Hotel,Pizza Place


## Cluster Postal codes

Run _k_-means to cluster the neighborhood into 3 clusters.


In [34]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Postal code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each postal code.

In [35]:
# add clustering labels
postalcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = downTor

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(postalcode_venues_sorted.set_index('Postal code'), on='Postal code')

toronto_merged.head() # check the last columns!

,index,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Theater,Pub,Breakfast Spot,Restaurant,Electronics Store,Event Space
1,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Restaurant,Park,Music Venue,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
2,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Hotel,Pizza Place
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,American Restaurant,Gastropub,Cocktail Bar,Italian Restaurant,Moroccan Restaurant,Cosmetics Shop,Creperie,Lingerie Store
4,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Pharmacy,Farmers Market,Seafood Restaurant,Cheese Shop,Restaurant,Beer Bar,Comfort Food Restaurant


Finally, let's visualize the resulting clusters

In [36]:
# create map
map_clusters = folium.Map(location=[latitudeDT, longitudeDT], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

**Cluster 1** - Sectors or neighborhoods focused on commerce and leisure / entertainment


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,-79.360636,0,Coffee Shop,Bakery,Park,Café,Theater,Pub,Breakfast Spot,Restaurant,Electronics Store,Event Space
1,M7A,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Restaurant,Park,Music Venue,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
2,M5B,-79.378937,0,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Hotel,Pizza Place
3,M5C,-79.375418,0,Café,Coffee Shop,American Restaurant,Gastropub,Cocktail Bar,Italian Restaurant,Moroccan Restaurant,Cosmetics Shop,Creperie,Lingerie Store
4,M5E,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Pharmacy,Farmers Market,Seafood Restaurant,Cheese Shop,Restaurant,Beer Bar,Comfort Food Restaurant
5,M5G,-79.387383,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Thai Restaurant,Burger Joint,Comic Shop,Park
7,M5H,-79.384568,0,Coffee Shop,Café,Restaurant,Gym,Clothing Store,Bakery,Thai Restaurant,Deli / Bodega,Asian Restaurant,Pizza Place
8,M5J,-79.381752,0,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Fried Chicken Joint,Brewery,Sporting Goods Shop,Italian Restaurant,Scenic Lookout
9,M5K,-79.381576,0,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Bakery,Seafood Restaurant,Japanese Restaurant,Salad Place,Italian Restaurant
10,M5L,-79.379817,0,Coffee Shop,Restaurant,Café,Hotel,Gym,Italian Restaurant,American Restaurant,Deli / Bodega,Japanese Restaurant,Seafood Restaurant


**Cluster 2** - Sectors or neighborhoods outside the city



In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,M4W,-79.377529,1,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


**Cluster 3** - Sectors or neighborhoods outside the city

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M6G,-79.422564,2,Grocery Store,Café,Park,Baby Store,Nightclub,Coffee Shop,Restaurant,Italian Restaurant,Athletics & Sports,Candy Store
